In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ESN
import pandas as pd

In [2]:
def set_seed(seed=42):
    """Making the seed (for random values) variable if None"""

    # Set the seed
    if seed is None:
        import time
        seed = int((time.time()*10**6) % 4294967295)
    try:
        np.random.seed(seed)
    except Exception as e:
        print( "!!! WARNING !!!: Seed was not set correctly.")
        print( "!!! Seed that we tried to use: "+str(seed))
        print( "!!! Error message: "+str(e))
        seed = None
    print( "Seed used for random values:", seed)
    return seed

In [5]:
import pandas as pd
dfs= []
for i in range(1,26):
    data = pd.read_csv(r'C:\Users\INFO-DSK-02\Desktop\Lorentz Multi Dimension Prediction-Phase-2\Final_Version\3D_ReservoirComputing\Input\High_Dimensional_Data\{}.txt'.format(i), header = None,sep="\t")
    data.drop(0, axis = 1, inplace = True)
    data.rename(columns = {1: 'x{}'.format(i)}, inplace = True)
    dfs.append(data)
df = pd.concat(dfs, axis = 1)
df = df.dropna()

In [6]:
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,176.598775,175.998775,172.2,172.0,171.25,169.5,169.0,155.682600,151.627000,150.327000,...,118.582215,112.075583,100.614083,94.156908,89.573324,89.281630,85.278315,81.069259,78.477433,76.218042
1,172.376055,171.776055,173.2,172.0,171.25,169.5,169.0,157.301557,153.020241,151.720241,...,119.997323,113.433513,101.904843,95.347797,90.706940,90.324103,86.250946,82.008262,79.370272,77.057304
2,172.657570,172.057570,173.2,173.0,172.25,170.5,170.0,157.645460,153.577537,152.277537,...,120.301768,113.725656,102.176182,95.598141,90.945245,90.548379,86.460197,82.210278,79.562356,77.237862
3,174.909687,174.309687,174.2,174.0,172.25,170.5,170.0,158.396198,154.134834,152.834834,...,120.973840,114.370573,102.785935,96.160715,91.480763,91.043477,86.922125,82.656236,79.986388,77.636449
4,176.880290,176.280290,175.2,174.0,173.25,171.5,171.0,158.983253,154.970778,153.670778,...,121.501313,114.876733,103.275709,96.612594,91.910910,91.432052,87.284667,83.006243,80.319189,77.949279


In [7]:
initLen = 1000
trainLen = initLen + 6000
testLen = 1900

In [39]:
data_in = df.loc[:,'x1':'x25']

In [40]:
data_in = np.array(data_in)

In [41]:
train_in = np.array(data_in[0:trainLen])
train_out = np.array(data_in[0+10:trainLen+10])
valid_in = np.array(data_in[trainLen:trainLen+testLen])
valid_out = np.array(data_in[trainLen+10:trainLen+testLen+10])

In [42]:
valid_in.shape

(1900, 25)

In [36]:
n_r = [300,400,500,800] # number of recurrent units
l_r =[ 0.3,0.2,0.1,0.4] # leaking rate (=1/time_constant_of_neurons)
s_r = [1,1.2,1.4,1.5,1.6]
r_c = [1e-3,1e-2,1e-1]
train_in = train_in
train_out = train_out
valid_in = valid_in
valid_out = valid_out
input_bias = True
dim_inp = 25
n_outputs = 25
initLen = initLen

In [37]:
def ESN_TUNE(n_reservoir, leak_rate, regularization_coef, spectral_radius, n_inputs,n_outputs, initLen, train_in, train_out, valid_in, valid_out):
    Best_Parameters = []
    input_bias = True
    nr = n_reservoir
    l_r = leak_rate
    r_c = regularization_coef
    s_r = spectral_radius
    dim_inp = n_inputs
    n_outputs = n_outputs
    dim_inp = n_inputs
    initLen = initLen
    for N in n_r:
        for L in l_r:
            for R in r_c:
                for i in range(len(s_r)):
                    parameters = []
                    N =  N#100
                    spectral_radius = s_r[i]
                    input_scaling = 1. # Scaling of input matrix
                    proba_non_zero_connec_W = 0.2 # Sparsity of recurrent matrix: Perceptage of non-zero connections in W matrix
                    proba_non_zero_connec_Win = 1. # Sparsity of input matrix
                    proba_non_zero_connec_Wfb = 1.
                        ### Generating random weight matrices with custom method
                    W = np.random.rand(N,N) - 0.5
                    if input_bias:
                        Win = np.random.rand(N,dim_inp+1) - 0.5
                    else:
                        Win = np.random.rand(N,dim_inp) - 0.5
                    Wfb = np.random.rand(N,n_outputs) - 0.5
                    mask = np.random.rand(N,N) # create a mask Uniform[0;1]
                    W[mask > proba_non_zero_connec_W] = 0 # set to zero some connections given by the mask
                    mask = np.random.rand(N,Win.shape[1])
                    Win[mask > proba_non_zero_connec_Win] = 0
                    Win = Win * input_scaling
                    original_spectral_radius = np.max(np.abs(np.linalg.eigvals(W)))
                    W = W * (spectral_radius / original_spectral_radius)
                    reservoir = ESN.ESN(lr=L, W=W, Win=Win, input_bias=input_bias, ridge=R, Wfb=None, fbfunc=None)
                    internal_trained = reservoir.train(inputs=[train_in], teachers=[train_out], wash_nr_time_step=initLen, verbose=False)
                    output_pred, internal_pred = reservoir.run(inputs=[valid_in,], reset_state=False)
                    df_pred = pd.DataFrame(output_pred[0])
                    df_test_out = pd.DataFrame(valid_out)
                    X_MSE = np.mean((df_test_out[0][:] - df_pred[0])**2)
                    Y_MSE = np.mean((df_test_out[1][:] - df_pred[1])**2)
                    Z_MSE = np.mean((df_test_out[2][:] - df_pred[2])**2)
#                     print('For ', N,L,R,s_r[i], 'parameters:: ', 'X, Y, Z MSE = ', X_MSE, Y_MSE, Z_MSE)
#                     print('RMSE = ', np.sqrt(X_MSE**2+Y_MSE**2+Z_MSE**2))
                    RMSE = np.sqrt(X_MSE**2+Y_MSE**2+Z_MSE**2)
                    parameters.append({'n_reservoir':N,'leaky_rate':L,'regularization_coef':R,'spectral_radius' :s_r[i]})
                    Best_Parameters.append([parameters, RMSE, X_MSE])
    df = pd.DataFrame(Best_Parameters)
    return df.iloc[np.argmin(np.array(df[1]))][0][0]

In [45]:
import time
t1 = time.perf_counter()
best_parameters = ESN_TUNE(n_reservoir= [300,400,500,800,1000],
         leak_rate = [ 0.3,0.2,0.1], 
         regularization_coef = [1e-3,1e-2,1e-1],
         spectral_radius = [1,1.2,1.4,1.5], 
         n_inputs =25,
         n_outputs = 25, 
         initLen  = initLen, 
         train_in = train_in, 
         train_out = train_out, 
         valid_in = valid_in, 
         valid_out = valid_out)
t2 = time.perf_counter()
print(t2-t1)

449.8826544000003


In [46]:
print('Best Parameters :', best_parameters)

Best Parameters : {'n_reservoir': 300, 'leaky_rate': 0.3, 'regularization_coef': 0.1, 'spectral_radius': 1.2}
